In [1]:
from selenium import webdriver
import time
from webdriver_manager.chrome import ChromeDriverManager
import re
import pandas as pd
import numpy as np
from selenium.webdriver.common.action_chains import ActionChains

In [47]:
factions = ['Breakers', 'Cave Goblins', 'Fallen Kingdom', 'Polar Dwarves', 'Savanna Elves', 'Vanguards',
            'Phoenix Elves', 'Tundra Orcs', 'Cloaks', 'Skyspear Avians', 'Fungal Dwarves', 'Eternal Council',
            'Obsidian Dwarves', 'Sand Goblins', 'Shadow Elves', 'Wayfarers', 'High Elves', 'Swamp Orcs',
             'Crimson Order', 'Mountain Vargath']

In [6]:
def login(driver):
    df = pd.read_excel('../sw_alt_password.xlsx')

    password = df['password'][1]
    email = df['Email'][1]

    url = 'https://www.plaidhatgames.com/accounts/login/?next=/redirect-swo'
    driver.get(url)


    time.sleep(5)
    login = driver.find_element_by_id('id_login')
    login.click()
    login.send_keys(email)
    time.sleep(.5)
    pw = driver.find_element_by_id('id_password')
    pw.click()
    pw.send_keys(password)
    time.sleep(.05)
    button = driver.find_element_by_class_name('primaryAction')
    button.click()
    time.sleep(15)


In [7]:
def get_card_attributes(driver, factions):
    login(driver)
    buttons = driver.find_elements_by_class_name('menu-button-text-container')
    buttons[3].click()
    time.sleep(3)
    card_data = []
    card_name_list = []
    for faction in factions:
        # select faction
        faction_button = driver.find_element_by_css_selector(f'[alt="{faction}"]')
        faction_button.click()
        # click continue
        buttons = driver.find_elements_by_class_name('menu-button-text-container')
        buttons[1].click()
        time.sleep(1)
        
        cards = driver.find_elements_by_class_name('card-image')
        cards[0].click()
        card_info, card_name_list = get_card_attributes_zoomed_in_card(driver, card_name_list)
        card_info.append(faction)
        card_info.append(1)
        card_data.append(card_info)
        
        time.sleep(.05)
        
        cards = driver.find_elements_by_class_name('card-image')
        cards[1].click()
        card_info, card_name_list = get_card_attributes_zoomed_in_card(driver, card_name_list)
        card_info.append(faction)
        card_info.append(2)
        card_data.append(card_info)
        time.sleep(.05)
        
        cards = driver.find_elements_by_class_name('card-image')
        cards[2].click()
        card_info, card_name_list = get_card_attributes_zoomed_in_card(driver, card_name_list)
        card_info.append(faction)
        card_info.append(5)
        card_data.append(card_info)
        time.sleep(.05)
        
        cards = driver.find_elements_by_class_name('card-image')
        cards[3].click()
        card_info, card_name_list = get_card_attributes_zoomed_in_card(driver, card_name_list)
        card_info.append(faction)
        card_info.append(5)
        card_data.append(card_info)
        time.sleep(.05)
        
        
        
        card_counts_in_faction = []
        card_counts = driver.find_elements_by_class_name('card-quantity')
        for count in card_counts:
            if 'x0' not in count.get_attribute('outerHTML'):
                card_counts_in_faction.append(count)
        for card_count in card_counts_in_faction:
            ac = ActionChains(driver)
            ac.move_to_element(card_count).move_by_offset(-20, 0).click().perform()
            card_info, card_name_list = get_card_attributes_zoomed_in_card(driver, card_name_list)
            # as long as card has not already been added
            if card_info:
                # obtain the card count
                html_card_count = re.findall('x[^<]+</div>',card_count.get_attribute('outerHTML'))
                regex_card_count = re.compile('{}(.*){}'.format(re.escape('x'), re.escape('</div>')))
                count = regex_card_count.findall(html_card_count[0])
                count = count[0]
                # append faction and card count
                card_info.append(faction)
                card_info.append(count)
                # append the card info as long as it's a new card
                card_data.append(card_info)
            time.sleep(.05)
            
            
        # go back
        buttons = driver.find_elements_by_class_name('menu-button-text-container')
        buttons[2].click()
        time.sleep(1)
        
    return card_data

In [40]:
def get_card_attributes_zoomed_in_card(driver, card_name_list):
    zoom_html = driver.find_element_by_class_name('zoom-card').get_attribute('outerHTML')
    
    html_card_name = re.findall('<div class="zoom-card-name-full">[^<]+</div>',zoom_html)
    regex_card_name = re.compile('{}(.*){}'.format(re.escape('<div class="zoom-card-name-full">'), re.escape('</div>')))
    card_name = regex_card_name.findall(html_card_name[0])
    card_name = card_name[0]
    print(card_name)
    if card_name not in card_name_list:
        card_name_list.append(card_name)
        
        if 'zoom-card-ability-name' in zoom_html:
            html_card_abilities = re.findall('<div class="zoom-card-ability-name">[^<]+</div>',zoom_html)
            regex_card_abilities = re.compile('{}(.*){}'.format(re.escape('<div class="zoom-card-ability-name">'), re.escape('</div>')))
            card_abilities = [regex_card_abilities.findall(html_card_abilities[i])[0] for i in range(len(html_card_abilities))]
            html_card_abilities_text = re.findall('<div class="zoom-card-ability-text"><span>[^<]+</span>',zoom_html)
            regex_card_abilities_text = re.compile('{}(.*){}'.format(re.escape('<div class="zoom-card-ability-text"><span>'), re.escape('</span>')))
            card_abilities_text = [regex_card_abilities_text.findall(html_card_abilities_text[i])[0] for i in range(len(html_card_abilities_text))]

        else:
            card_abilities = None
            card_abilities_text = None
            
        card_classes = []
        
        if 'zoom-card-extras-unit' in zoom_html:   
            html_card_classes = re.findall('<div class="zoom-card-extras-unit">[^<]+</div>',zoom_html)
            regex_card_classes = re.compile('{}(.*){}'.format(re.escape('<div class="zoom-card-extras-unit">'), re.escape('</div>')))
            card_classes = regex_card_classes.findall(html_card_classes[0])
        if 'Structure' in zoom_html:
            card_classes.append('Structure')
            


        if ('place ' in zoom_html or 'Place ' in zoom_html) and ' under ' in zoom_html:
            go_under = True
        else:
            go_under = False
            
        if 'event-text-active' in zoom_html:
            active_event = True
        else:
            active_event = False
            
        if 'zoom-card-extras-event-phase' in zoom_html and 'Structure' not in zoom_html:
            html_event_phase = re.findall('<div class="zoom-card-extras-event-phase">[^<]+</div>',zoom_html)
            regex_event_phase = re.compile('{}(.*){}'.format(re.escape('<div class="zoom-card-extras-event-phase">'), re.escape('</div>')))
            event_phase = regex_event_phase.findall(html_event_phase[0])
            event_phase = event_phase[0]
        else:
            event_phase = None
            
            
        

        
            
        if 'zoom-card-cost' in zoom_html:
            html_card_cost = re.findall('<div class="zoom-card-cost[^"]*">[^<]+</div>',zoom_html)
            regex_card_cost = re.compile('{}(.*){}'.format(re.escape('>'), re.escape('</div>')))
            card_cost = regex_card_cost.findall(html_card_cost[0])
            card_cost = card_cost[0]
        else:
            card_cost = None
            
        
            
        
        # so only units are selected (for whatever reason events have an empty strength div)
        if 'zoom-card-extras-unit' in zoom_html:
            html_card_strength = re.findall('<div class="zoom-card-strength">[^<]+</div>',zoom_html)
            regex_card_strength = re.compile('{}(.*){}'.format(re.escape('<div class="zoom-card-strength">'), re.escape('</div>')))
            card_strength = regex_card_strength.findall(html_card_strength[0])
            card_strength = card_strength[0]
        else:
            card_strength = None
        
        # only units and structures should have a life value
        if 'zoom-card-extras-unit' in zoom_html or 'Structure' in zoom_html:
            html_card_life = re.findall('<div class="zoom-card-life[^"]*">[^<]+</div>',zoom_html)
            regex_card_life = re.compile('{}(.*){}'.format(re.escape('>'), re.escape('</div>')))
            card_life = regex_card_life.findall(html_card_life[0])
            card_life = card_life[0]
        else:
            card_life = None



        elem = driver.find_element_by_id('zoom-card')
        ac = ActionChains(driver)
        ac.move_to_element(elem).move_by_offset(300, 400).click().perform()
        time.sleep(.05)

        return [card_name, card_classes, event_phase, active_event, card_abilities, card_abilities_text, 
               go_under, card_cost, card_strength, card_life], card_name_list
    
    else:
        elem = driver.find_element_by_id('zoom-card')
        ac = ActionChains(driver)
        ac.move_to_element(elem).move_by_offset(300, 400).click().perform()
        time.sleep(.05)

        return False, card_name_list

In [89]:
driver = webdriver.Chrome(executable_path=r"C:\\Users\\steve\\.wdm\\drivers\\chromedriver\\win64\\119.0.6045.105\\chromedriver.exe")
# use this when you need a new version of chromedriver
# driver = webdriver.Chrome(ChromeDriverManager().install())
card_data = get_card_attributes(driver, factions)
card_data

Tacullu
Mind Control
Deceiver
Mind Witch
Gulldune
Gwalark
Kalal
Deceiver
Mind Witch
Wind Archer
Wind Mage
Blast
Hypnotic Call
Perplexing Tempest
Sneeks
Enrage the Horde
Horde Slinger
Beast Rider
Blarf
Smeg
The Eater
Beast Rider
Clinger
Horde Climber
Horde Slinger
Pile On
Sneak
Unrelenting
Ret-Talus
Sacrificial Pyre
Undead Carrier
Undead Archer
Dragos
Elut-Bal
Gul-Dass
Hellfire Cultist
Undead Archer
Undead Carrier
Undead Warrior
Blood Summon
Hellforged Weapon
Purge
Svara
Ice Ram
Frost Mage
Ice Golem
Jarmund
Nadiana
Ollag
Bear Cavalry
Frost Mage
Ice Golem
Ice Smith
Glacial Shift
Ice Repair
Parapet
Abua Shi
Chant of Power
Border Archer
Lioness
Makeinda Ru
Miti Kyru
Miti Mumway
Border Archer
Lioness
Rhinoceros
Spirit Mage
Chant of Entangling
Chant of Growth
Chant of Weaving
Sera Eldwyn
Holy Judgment
Temple Priest
Citadel Knight
Colleen Brighton
Jacob Eldwyn
Valna Stoutheart
Citadel Archer
Citadel Knight
Citadel Paladin
Temple Priest
Guarding Spirits
Mass Heal
Renewed Hope
Queen Maldaria
Sp

[['Tacullu',
  ['Summoner'],
  None,
  False,
  ['Mind Capture'],
  ['When this unit attacks an enemy unit, if the damage added would be enough to destroy the target, you may instead ignore that damage and take control of the target.'],
  False,
  None,
  '3',
  '13',
  'Breakers',
  1],
 ['Mind Control',
  [],
  'Summon Phase',
  False,
  None,
  None,
  False,
  '0',
  None,
  None,
  'Breakers',
  2],
 ['Deceiver',
  ['Common'],
  None,
  False,
  ['Stupify', 'Engage'],
  ['When an adjacent enemy attacks any card, if 1 or more ',
   'Each time an adjacent enemy unit moves or is forced away from this unit, add 1 damage to that enemy.'],
  False,
  '1',
  '1',
  '4',
  'Breakers',
  5],
 ['Mind Witch',
  ['Common'],
  None,
  False,
  ['Mimic'],
  ['At the start of your Move Phase, this unit may target a common within 3 spaces. This unit has the target’s abilities until the end of your turn.'],
  False,
  '1',
  '3',
  '2',
  'Breakers',
  5],
 ['Gulldune',
  ['Champion'],
  None,
  F

In [50]:
# card_df.to_csv('Summoner_Wars_Cards.csv')